## install mecab on mac
```
brew install mecab mecab-ipadic  
pip install mecab-python3
```

## install neolog
```
git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd
./bin/install-mecab-ipadic-neologd -n
```

## get Japanese articles
livedoor ニュースコーパス

```
wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
tar xvzf ldcc-20140209.tar.gz
```

In [1]:
import MeCab
from urllib import request 
from pathlib import Path
from gensim import corpora, models
import numpy as np
import tqdm

In [2]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [10]:
[line.split("\t") for line in mecab.parse("今日も1日がんばるぞい！").split("\n")]

[['今日', 'キョウ', '今日', '名詞-副詞可能', '', ''],
 ['も', 'モ', 'も', '助詞-係助詞', '', ''],
 ['1日', 'ツイタチ', '1日', '名詞-固有名詞-一般', '', ''],
 ['がん', 'ガン', 'がん', '名詞-一般', '', ''],
 ['ばる', 'バル', 'バル', '名詞-一般', '', ''],
 ['ぞい', 'ゾイ', 'ぞい', '名詞-接尾-一般', '', ''],
 ['！', '！', '！', '記号-一般', '', ''],
 ['EOS'],
 ['']]

In [4]:
doc_dir = Path("./text/")
dirs = [i for i in doc_dir.iterdir() if i.is_dir()]
dirs

[PosixPath('text/dokujo-tsushin'),
 PosixPath('text/it-life-hack'),
 PosixPath('text/kaden-channel'),
 PosixPath('text/livedoor-homme'),
 PosixPath('text/movie-enter'),
 PosixPath('text/peachy'),
 PosixPath('text/smax'),
 PosixPath('text/sports-watch'),
 PosixPath('text/topic-news')]

In [5]:
articles = [a for categ in dirs for a in categ.iterdir()]

In [6]:
len(articles)

7376

In [66]:
def read_doc(doc_id):
    with articles[doc_id].open() as f:
        print(f.read())

In [7]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]

In [45]:
def tokenizer(text):
    l = [line.split("\t") for line in mecab.parse(text).split("\n")]
    res = [i[0] for i in l 
                   if len(i) >=4 
                       and ("名詞" in i[3] )#or "動詞" in i[3] or "形容詞" in i[3] )
                       and "数" not in i[3] and "助動詞" not in i[3] 
                       and i[0] not in stopwords
            ]
    return res
tokenizer("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['自分自身', '若さ故の過ち']

In [11]:
docs = []
for a in tqdm.tqdm(articles):
    with a.open() as f:
        f.readline()
        f.readline()
        docs.append(tokenizer(f.read()))

100%|██████████| 7376/7376 [00:30<00:00, 241.02it/s]


In [12]:
docs[0][:10]

['友人', '代表', 'スピーチ', '独女', 'こなし', 'いる', 'ジューン・ブライド', '呼ば', 'れる', '６月']

In [13]:
no_below = 5
no_above = 0.2
d = corpora.Dictionary(docs)
d.filter_extremes(no_below, no_above)
d.compactify()

In [14]:
dic_num = len(d)
dic_num

26209

In [15]:
d.doc2bow(docs[0][:10])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]

(id, count)

In [16]:
d[7] # get word from id

'お祝い'

In [55]:
corpus = [d.doc2bow(w) for w in docs]

In [56]:
corpus_words = sum(count for doc in corpus for id, count in doc)
corpus_words

1496217

In [64]:
num_topics = 100

In [65]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=num_topics, iterations=100, passes=5)

In [68]:
lda.print_topics(5)

[(83,
  '0.090*"MAX" + 0.088*"S" + 0.064*"エスマックス" + 0.030*"smaxjp" + 0.028*"関連リンク" + 0.024*"on" + 0.019*"ライブドアブログ" + 0.018*"向け" + 0.018*"執筆" + 0.014*"Twitter"'),
 (73,
  '0.045*"離婚" + 0.018*"浮気" + 0.016*"週間ランキング" + 0.016*"2位" + 0.015*"1位" + 0.015*"3位" + 0.013*"原因" + 0.011*"プログラム" + 0.010*"手術" + 0.010*"クレンジング"'),
 (87,
  '0.034*"B" + 0.030*"ファイル" + 0.029*"データ" + 0.027*"擁護" + 0.021*"ダウンロード" + 0.017*"コピー" + 0.017*"動画" + 0.016*"共有" + 0.015*"C" + 0.014*"A"'),
 (38,
  '0.082*"転職" + 0.029*"求人" + 0.028*"play" + 0.022*"google" + 0.021*"livedoor" + 0.018*"パス" + 0.016*"求人情報" + 0.015*"マッチ" + 0.015*"判断" + 0.015*"type"'),
 (97,
  '0.032*"声" + 0.031*"批判" + 0.023*"発言" + 0.017*"コメント" + 0.016*"寄せ" + 0.016*"報じ" + 0.014*"番組" + 0.012*"てる" + 0.012*"意見" + 0.012*"放送"')]

## see on tensorboard

http://projector.tensorflow.org/  
ref: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Tensorboard_visualizations.ipynb


In [ ]:
all_topics = lda.get_document_topics(corpus, minimum_probability=0)
all_topics[0]

In [ ]:
with open('doc_lda_tensor.tsv','w') as w:
    for doc_topics in all_topics:
        for topics in doc_topics:
            w.write(str(topics[1])+ "\t")
        w.write("\n")    

In [ ]:
meta = [str(a).split("/") for a in articles]

In [ ]:
meta[0]

In [ ]:
with open('doc_lda_metadata.tsv','w') as w:
    w.write('Titles\tGenres\n')
    for m in meta:
        w.write("%s\t%s\n" % (m[1][:2], m[1]))

## perplexity

In [ ]:
lda.log_perplexity(corpus)

Estimate the variational bound of documents from corpus: E_q[log p(corpus)] - E_q[log q(corpus)]

In [ ]:
bound = lda.bound(corpus)/corpus_words
bound #= lda.log_perplexity(corpus)

In [ ]:
import numpy as np

In [ ]:
perplexity = np.exp2(-bound ) # 2^(-bound per words)

In [ ]:
lda.top_topics(corpus=corpus, num_words=10)

## 類似度

In [19]:
#corpora.BleiCorpus.serialize("./corpus.blei", corpus)
#blei_corpus = corpora.BleiCorpus('./corpus.blei', './corpus.blei.vocab')
#model = models.ldamodel.LdaModel(corpus,num_topics=100,id2word=corpus.id2word)

In [82]:
doc_topics = [lda[c] for c in corpus]

In [83]:
doc_topics[0]

[(1, 0.024539963729934569),
 (4, 0.22117063381154689),
 (13, 0.018797915308231414),
 (22, 0.015263599413199113),
 (30, 0.22314544956859694),
 (36, 0.010602068241370506),
 (41, 0.051402785541291858),
 (47, 0.042586155933477775),
 (55, 0.061647037899655185),
 (57, 0.10323390553823665),
 (75, 0.040703228018363902),
 (91, 0.083498075042254052),
 (97, 0.044532086413806092),
 (99, 0.04200734705002112)]

In [84]:
len(doc_topics) #  = len(article)

7376

$$
\text{doc_id(7376)}
\underbrace{
\begin{pmatrix}
0 & 0.024539963729934569 & 0 & \cdots & 0\\
0 & 0 & 0 & \cdots & 0\\
0 & 0 & 0 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 0
\end{pmatrix}
}_{\text{topics(100)}}
$$

In [88]:
dense = np.zeros( (len(doc_topics), num_topics), float)
for doc_id, t in enumerate(doc_topics):
    for topic_id, weight in t:
        dense[doc_id, topic_id] = weight

In [100]:
from scipy.spatial import distance
pairwise = distance.squareform(distance.pdist(dense))

In [122]:
def similar_to(doc_id, num=5, distance=False):
    doc = pairwise[doc_id]
    ids = np.argsort(doc)[1: num+1] # most similar is self.
    if distance:
        return [(x,y) for x, y in zip(ids,  np.sort(x)[1: num+1])]
    else:
        return ids

In [123]:
similar_to(0, distance=True)

[(730, 0.20323303971602655),
 (149, 0.23860101443188048),
 (325, 0.24659415533439674),
 (2693, 0.27762505985769631),
 (2690, 0.2805878942847832)]

In [125]:
read_doc(0)

http://news.livedoor.com/article/detail/4778030/
2010-05-22T14:30:00+0900
友人代表のスピーチ、独女はどうこなしている？
　もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……という「お祝い貧乏」状態の人も多いのではないだろうか？　さらに出席回数を重ねていくと、こんなお願いごとをされることも少なくない。

　「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」

　さてそんなとき、独女はどう対応したらいいか？

　最近だとインターネット等で検索すれば友人代表スピーチ用の例文サイトがたくさん出てくるので、それらを参考にすれば、無難なものは誰でも作成できる。しかし由利さん（33歳）はネットを参考にして作成したものの「これで本当にいいのか不安でした。一人暮らしなので聞かせて感想をいってくれる人もいないし、かといって他の友人にわざわざ聞かせるのもどうかと思うし……」ということで活用したのが、なんとインターネットの悩み相談サイトに。そこに作成したスピーチ文を掲載し「これで大丈夫か添削してください」とメッセージを送ったというのである。

　「一晩で3人位の人が添削してくれましたよ。ちなみに自分以外にもそういう人はたくさんいて、その相談サイトには同じように添削をお願いする投稿がいっぱいありました」（由利さん）。ためしに教えてもらったそのサイトをみてみると、確かに「結婚式のスピーチの添削お願いします」という投稿が1000件を超えるくらいあった。めでたい結婚式の影でこんなネットコミュニティがあったとは知らなかった。

　しかし「事前にお願いされるスピーチなら準備ができるしまだいいですよ。一番嫌なのは何といってもサプライズスピーチ！」と語るのは昨年だけで10万以上お祝いにかかったというお祝い貧乏独女の薫さん（35歳）

　「私は基本的に人前で話すのが苦手なんですよ。だからいきなり指名されるとしどろもどろになって何もいえなくなる。そうすると自己嫌悪に陥って終わった後でもまったく楽しめなくなりますね」
　
　サプライズスピーチのメリットとしては、準備していない状態なので、フランクな本音をしゃべってもらえるという楽しさがあるようだ。しかしそれも上手に対応できる人なら

In [126]:
read_doc(730)

http://news.livedoor.com/article/detail/6528405/
2012-05-04T10:00:00+0900
なぜモテない？！料理男子
このところ、速水もこみちや向井理など料理上手なイメケンが人気だ。爽やかな笑顔で手際よく料理をする様子を見ていると「料理ができる男性って素敵！」と思ってしまうのだが…。身近な料理男子たちはモテとは無縁らしい。それはなぜ？
家電メーカーに勤務する涼さん（27歳）は、弟と二人暮らし。節約も兼ねて二人で交互に弁当や夕食を作っているせいか、その腕前もなかなかのもの。「…だからといって僕も弟もモテることはありません。同僚の女性たちと一緒に弁当を食べていますが、僕が料理の話を始めると軽くスルーされます。料理って男女共通の話題だと思っていたんですが、そうでもないようです」と話してくれた。

身近な独女たちに涼さんの話をしてみると「料理をする男性はウンチクが多いから、スルーしたくなる気持ちは分かる」「女性だからといって、みんなが料理に関心があるわけではないから話を振られても困ると思う」といった意見が飛び出した。涼さんは料理の腕前を自慢するようなタイプではないのだが、ちょっとした知識が女性たちには自慢に聞こえてしまうのかもしれない。

「ついつい“こだわり”を語って失敗しています」と話してくれたのは、自炊歴10年の健一さん（28歳・研究職）。「自分で炊いたご飯と初めて焼いたシャケの美味しさで料理に目覚めました。理系のせいか調味料はレシピ通りにキチっと測ります。そうすると大抵のものは美味しくできるんですよ」と健一さん。ケーキやクッキーも得意だ。
「出汁は昆布と鰹節でとります。醤油は濃口と薄口を料理によって使い分けていますし、鍋類にもこだわりがあります。料理のことを話し始めると自分なりのこだわりを熱く語っちゃうんですよ。案外、男同士だと意気投合するんですけど、女性には確実にドン引かれます（苦笑）」

健一さんの後輩にも料理上手な男性が増えている。料理好きになったきっけは「自分で肉を焼いたら、すごく美味しかった」「自分でケーキを焼いたら美味しくできた」など、自分で作れた→美味しい→俺って料理の才能があるかも！といった場合が多いらしい。また「両親が共働きだったため、晩ごはんを作り始めたら家族に好評だった」という男性も。

